In [3]:
from math import *
from scipy import optimize
import numpy as np
from scipy import *
import scipy.linalg
import scipy.interpolate

def bounded_residuals(z, w, p, model, bounds = -1, weight = -1):
    rednum = 1.e1
    if not ilen(p):
        p = [p]
    if bounds == -1 or not ilen(p) == ilen(bounds):
        bounds = [[0,0] for i in range(len(p))]
    weight = 1. if ilen(weight) == 1 else weight
    if within_bounds(p, bounds):
        return (model(z, p) - w)/weight
    else:
        if not ilen(z):
            return rednum*(model(z,p) - w)/weight
        else:
            return rednum*(model(z,p) - w)/weight

def fitfunc(x, y, func, params, bounds = -1, weights = -1, full_output = False):
    x = array(x)
    y = array(y)
    weights = np.array([1. for i in range(len(x))]) if ilen(weights) == 0 else weights
    errfunc = lambda p, z, w, wght: bounded_residuals(z, w, p, func, bounds = bounds, weight = wght)
    if not full_output:
        p1, success = optimize.leastsq(errfunc, params[:], args=(x,y,weights), full_output=False)
        if not success:
            print ('Fit failed')
            return [0 for i in range(len(params))]
        return p1
    else:
        p1, cov, retval, mesg, info = optimize.leastsq(errfunc, params[:], args=(x,y,weights), full_output=True)
        resvar = np.array(np.array(residuals(x, y, func, p1))/weights)**2
        resvar = (resvar.sum()/(len(resvar) - len(p1)))
        cov = np.matrix(cov)*resvar if not cov == None else None
        return p1, cov

def within_bounds(p, b):
    if not ilen(p):
        p = [p]
        b = [b]
    counter = -1
    value = True
    for pa in p:
        counter += 1
        if max(b[counter]) == min(b[counter]):
            value *= True
        elif pa < max(b[counter]) and pa > min(b[counter]):
            value *= True
        else:
            value *= False
    return value

def ilen(a):
    try:
        n = len(a)
    except:
        n = 0
    return n

#Needed code will look like below

def test(x, a): #Define the function that you want to fit to
    x = np.array(x)
    return a*x**2

x = np.arange(0,10,0.01) #Data independent variable
data = test(x,6.5) + .1*np.random.rand(len(x)) #Data dependent variable (with noise)
fits = fitfunc(x, data, test, [1.]) #Finding the fits for the non-data parameters in test. If test has more than one functional parameter, say coefficients to a polynomial or whatnot, the code will find the fits for all of the parameters
print (fits)

[ 6.50086259]
